In [28]:
import pickle
import matplotlib as plt
import seaborn as sns

ImportError: No module named matplotlib

In [11]:
with open("./p5/final_project/final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

In [22]:
features = dict()
for k, v in data_dict.iteritems():
    for k2, v2 in v.iteritems():
        features[k2] = v2

In [24]:
for k, v in features.iteritems():
    pd.hist(k, v)

AttributeError: 'module' object has no attribute 'hist'